'/content'

In [ ]:
# Import relevant libraries to make HTTP requests and parse JSON response
import requests
import json
import csv
import io
import chembl_webresource_client as cwc
import pandas as pd
import pickle
import os

print("done")

done


In [ ]:
# Set disease_id variable for desired disease
disease_id = "EFO_0005537"

# Build query string to get target information as well as count
query_string = """
query AssociatedTargets {
  disease(efoId: "disease_id") {
    id
    name
    associatedTargets(page: { size: 1, index: 2 }) {
      rows {
        target {
          id
          approvedName
          dbXrefs{
              id
            }
        }
        score
      }
    }
  }
}
""".replace("disease_id", disease_id)

# Set variables object of arguments to be passed to endpoint
variables = {"efoId": disease_id}

# Set base URL of GraphQL API endpoint
base_url = "https://api.platform.opentargets.org/api/v4/graphql"

# Perform POST request and check status code of response
r = requests.post(base_url, json={"query": query_string, "variables": variables})
print(r.status_code)

#Transform API response from JSON into Python dictionary and print in console
api_response = json.loads(r.text)
print('done')
#print(api_response)


200
done


In [ ]:
a=0
#create list of target IDs with associated evidence scores by calling them from dictionary
target_info = api_response['data']['disease']['associatedTargets']['rows']
#print(target_info)


target_id_list = []
#pull IDs and scores from dictionary and add to new list
while a < len(target_info):
    #print(a)

    for dict in target_info[a]['target']['dbXrefs']:
        #print(dict)
        # Get rid of all IDs that aren't CHEMBL IDs, then put CHEMBL ID in list to add to tuple
        newDict = {key:val for key, val in (dict).items() if "CHEMBL" in val}
        if len(newDict) == 0:
            continue
        chembl_id = newDict['id']
    target_id_list.append((target_info[a]['target'].get('id'), chembl_id, target_info[a]['score']))
    a+=1
print(target_id_list)

[('ENSG00000120217', 'CHEMBL3580522', 0.40556949639812895)]


In [ ]:
#wkdir = os.mkdir(disease_id + "targets info")
print(wkdir)
print(os.getcwd())


None
/content


In [ ]:
from chembl_webresource_client.new_client import new_client

target = new_client.target
activity = new_client.activity
assay = new_client.assay

# Create targets_dict, which will be a dictionary containing target names, with compounds and compound data associated with that target.
targets_dict = {}

# Create darget_data, which will be a dictionary containing target names, with dataframes with compound data associated with that target
target_data_list = []

#For each target pulled from OpenTargets:
for target in target_id_list:

    target_data = {}
    print(target[0])
    # Get compound activitiy data from Chembl for each target, including only exact IC50 values. Sort by IC50 value.
    activities = activity.filter(target_chembl_id=target[1], standard_type="IC50", relation = '=').order_by('standard_value')
    #print(assay)
    print(activities)

    compound_dictionary = {}

    #create seperate list for each type of data
    smiles_list = []
    standard_value_list = []
    compound_list = []
    type_list = []
    assays_list = []
    assay_type_list = []
    descr_list = []
    abstract_list = []
    doc_list = []

    print(len(activities))

    for compound in activities:
        #print(compound)
        slimDict = {key:val for key, val in compound.items() if key == 'type' or key == 'canonical_smiles' or key == 'standard_value'}


        #slimDict = {key:val for key, val in compound.items() if key == 'value'}


        compound_id = compound['molecule_chembl_id']
        compound_list.append(compound_id)
        type_list.append(compound['type'])
        standard_value_list.append(compound['standard_value'])
        smiles_list.append(compound['canonical_smiles'])
        assays_list.append(compound['assay_chembl_id'])
        assay_type_list.append(compound['assay_type'])
        doc_list.append(compound['document_chembl_id'])

        doc_id = compound['document_chembl_id']
        document = new_client.document
        abstract = document.filter(document_chembl_id = doc_id, assay_chembl_id = compound['assay_chembl_id']).only('abstract')
        print(abstract[0]['abstract'])
        abstract_list.append(abstract[0]['abstract'])
        descr_list.append(compound['assay_description'])
        #print(compound_id)

        compound_dictionary[compound['molecule_chembl_id']] = slimDict
        df = pd.DataFrame.from_dict(compound_dictionary)


    targets_dict[target[0]] = compound_dictionary


    to_pandas_dict = {"compound id": compound_list, "smiles": smiles_list, "type" : type_list, "standard value": standard_value_list, 'assay_id': assays_list, 'assay type': assay_type_list, 'assay description' : descr_list, 'document ID' : doc_list, 'abstract' : abstract_list}
    pandas_df = pd.DataFrame.from_dict(to_pandas_dict)
    """
    f = open(target[0]+"dataframe.pickle", "wb")
    pickle.dump(pandas_df, f)
    f.close()
    """
    display(pandas_df)
    target_data['target_id'] = target[0]
    target_data['data'] = pandas_df
    target_data_list.append(target_data)




"""
import pickle

for target in target_data_list:
    print(target['target_id'])

    display(target['data'])
    f = open(target['target_id']+"dataframe.pickle", "wb")
    pickle.dump(target['data'], f)
    f.close()
    #display(target['data'])
"""




ENSG00000120217
[{'action_type': {'action_type': 'INHIBITOR', 'description': 'Negatively effects (inhibits) the normal functioning of the protein e.g., prevention of enzymatic reaction or activation of downstream pathway', 'parent_type': 'NEGATIVE MODULATOR'}, 'activity_comment': None, 'activity_id': 22991297, 'activity_properties': [{'comments': None, 'relation': '=', 'result_flag': 0, 'standard_relation': '=', 'standard_text_value': None, 'standard_type': 'TIME', 'standard_units': 'hr', 'standard_value': '0.25', 'text_value': None, 'type': 'Time', 'units': 'min', 'value': '15.0'}], 'assay_chembl_id': 'CHEMBL4775268', 'assay_description': 'Binding affinity to human PDL1 assessed as reduction in PDL1/human PD1protein-protein interaction by HTRF binding assay', 'assay_type': 'B', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000357', 'bao_label': 'single protein format', 'canonical_smiles': 'Cc1c(COc2cc(OCc3cncc(C#N)c

,compound id,smiles,type,standard value,assay_id,assay type,assay description,document ID,abstract
0,CHEMBL4776444,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CNC(C)(C)C(=O)O)...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...
1,CHEMBL4081869,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CNCCNC(C)=O,IC50,1.0,CHEMBL4775269,B,Binding affinity to PDL1 (unknown origin) asse...,CHEMBL4765466,Therapeutic interference of the programmed cel...
2,CHEMBL4776700,Cn1cnc(S(=O)(=O)N2CCCC3=C[C@H]4C[C@H](CN5CCCC[...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...
3,CHEMBL4785255,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CN[C@](C)(CO)C(=...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...
4,CHEMBL5171101,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](C)...,IC50,1.03,CHEMBL5096762,B,Inhibition of PD-L1 (unknown origin) by HTRF a...,CHEMBL5096155,Tumor immunotherapy has made great progress in...
5,CHEMBL4071326,Cc1c(COc2cc(OCc3cccc(C#N)c3)c(CN3C[C@H](O)C[C@...,IC50,1.4,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...
6,CHEMBL4084368,Cc1cc(CN[C@H](CO)C(=O)O)c(OCc2cccc(C#N)c2)cc1O...,IC50,2.25,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...
7,CHEMBL4288470,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](CC...,IC50,5.2,CHEMBL4263391,B,Inhibition of PD-L1 (unknown origin),CHEMBL4261578,The modulation of immune checkpoint receptors ...
8,CHEMBL3582254,Cc1c(COc2ccc(CN[C@H](C)C(=O)O)cc2Cl)cccc1-c1cc...,IC50,6.0,CHEMBL3583018,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL3580542,
9,CHEMBL4061613,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CN[C@@H]...,IC50,6.0,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...


'\nimport pickle\n\nfor target in target_data_list:\n    print(target[\'target_id\'])\n\n    display(target[\'data\'])\n    f = open(target[\'target_id\']+"dataframe.pickle", "wb")\n    pickle.dump(target[\'data\'], f)\n    f.close()\n    #display(target[\'data\'])\n'

In [ ]:
from chembl_webresource_client.new_client import new_client

assay = new_client.document

abstract = assay.filter(assay_chembl_id = "CHEMBL4775268").only('abstract')
print(abstract)

[{'abstract': ''}, {'abstract': ''}, {'abstract': 'Okadaic acid (OA) is a toxin responsible for diarrhetic shellfish poisoning and is an extremely useful tool for studying processes that are regulated by phosphorylation, although the exact mechanism of action is still undetermined. We report on a study that proved the existence of OA in an unusual dimeric form when complexed with potassium ion. The proposed structure of this dimer is based on spectroscopic and conformational studies.'}, {'abstract': 'A variety of novel heterocyclic compounds having thienoazepine, pyrroloazepine, furoazepine, and thienodiazepine skeletons were synthesized, most of which exhibited potent antagonism of [(3)H]-AVP specific binding in assays using rat liver (V1), rat kidney (V2), human platelet plasma membranes, and recombinant human CHO cells (V2), as well as antagonizing AVP-induced hypertension in rats (V1, intravenous) and showing a diuretic effect in rats (V2, oral). By detailed studies of the structur

In [ ]:
big_target_data_list = [target for target in target_data_list if len(target['data']) > 3000]
print(type(target_data_list[0]['data']))
print(big_target_data_list)

<class 'pandas.core.frame.DataFrame'>
[]


In [ ]:
display(target_data_list[0]['data'])

,compound id,smiles,type,standard value,assay_id,assay type
0,CHEMBL4776444,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CNC(C)(C)C(=O)O)...,IC50,1.0,CHEMBL4775268,B
1,CHEMBL4081869,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CNCCNC(C)=O,IC50,1.0,CHEMBL4775269,B
2,CHEMBL4776700,Cn1cnc(S(=O)(=O)N2CCCC3=C[C@H]4C[C@H](CN5CCCC[...,IC50,1.0,CHEMBL4775268,B
3,CHEMBL4785255,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CN[C@](C)(CO)C(=...,IC50,1.0,CHEMBL4775268,B
4,CHEMBL5171101,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](C)...,IC50,1.03,CHEMBL5096762,B
5,CHEMBL4071326,Cc1c(COc2cc(OCc3cccc(C#N)c3)c(CN3C[C@H](O)C[C@...,IC50,1.4,CHEMBL4017391,B
6,CHEMBL4084368,Cc1cc(CN[C@H](CO)C(=O)O)c(OCc2cccc(C#N)c2)cc1O...,IC50,2.25,CHEMBL4017391,B
7,CHEMBL4288470,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](CC...,IC50,5.2,CHEMBL4263391,B
8,CHEMBL3582254,Cc1c(COc2ccc(CN[C@H](C)C(=O)O)cc2Cl)cccc1-c1cc...,IC50,6.0,CHEMBL3583018,B
9,CHEMBL4061613,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CN[C@@H]...,IC50,6.0,CHEMBL4017391,B


In [ ]:
# Get TXT file of list of SMILES associated with target
for target in target_data_list:
    print(target['target_id'])
    smiles = target['data']['smiles']
    print(smiles)
    smiles_list = []
    for row in smiles:
        smiles_list.append(row)
    print(smiles_list)

    smiles_string = str(smiles_list)
    smiles_string = smiles_string[1:-1]
    smiles_string = smiles_string.replace(",", "\n").replace("'", "").replace(" ", "")
    print(smiles_string)

    #with open(target['target_id']+'_Smiles.txt','w') as data:
        #data.write(str(smiles_string))
        # Get TXT file of list of SMILES associated with target


In [ ]:

smiles = "CC(O)C"
headers = 	{
	"API Key":"0794a3fb-40ff-4465-9b26-40bfad7ce84e",
	"Structure":smiles,
	"Search Type":'3',
	"Maximum Search Time":'60000',
	"Maximum Result Count":'10000',
	"Chemical Similarity Index":'0.9'
	}
url = "https://api.molport.com/api/chemical-search/search"

#r = requests.get("https://api.molport.com/api/molecule/load?molecule=2325030&apikey=880d8343-8ui2-418c-9g7a-68b4e2e78c8b")
r = requests.get(url = url, params = headers)
r.json()

{'Result': {'Status': 2, 'Message': 'Username or password is incorrect!'},
 'Data': {'Molecules': [], 'Version': 'v.1.0'}}

In [ ]:
r = requests.get("https://zinc20.docking.org/for_sale?CC(C)O")
print(r.status)

NameError: name 'compund_dictionary' is not defined